install.packages('data.table', repos = "http://cran.rstudio.com")

In [1]:
library(ggplot2)
library(dplyr) # data wrangling 
library(viridis) # colour blind friendly palette, works in B&W also
library(Interpol.T) #  will generate a large dataset on initial load
library(lubridate) # for easy date manipulation
library(ggExtra)
library(tidyr)
library(data.table)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: date
Loading required package: chron

Attaching package: ‘lubridate’

The following objects are masked from ‘package:chron’:

    days, hours, minutes, seconds, years

The following object is masked from ‘package:base’:

    date


Attaching package: ‘data.table’

The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year

The following objects are masked from ‘package:dplyr’:

    between, first, last



In [2]:
data = fread('/public/npatil4/networks/data/combined/decimal3/data_2015_withNodes_hour.txt',
                   sep='|')

Read 140909694 rows and 12 (of 12) columns from 11.042 GB file in 00:04:03


In [3]:
data1 <- data.frame(data)

In [4]:
rm(data)

In [7]:
df2 = data1
df2$year = 2015


In [8]:
df3 = df2 %>%
    group_by(year,month,day,hour)%>%
    summarise(
    average_trips = mean(trips, na.rm = TRUE),
    total_trips = sum(trips, na.rm = TRUE),
    average_charge = mean(sum(charge*trips)/sum(trips)),
    average_duration = mean(sum(duration*trips)/sum(trips))
  )

In [9]:
rm(data1)

In [10]:
df3$city = 'New York City'
df3 <-df3 %>%
            select(city,day,hour,month,year,average_trips) %>%
            fill(average_trips)

statno <-unique(df3$city)

In [11]:
head(df3)

,city,day,hour,month,year,average_trips
1,New York City,1,0,1,2015,1.01825341331469
2,New York City,1,1,1,2015,1.01604067574217
3,New York City,1,2,1,2015,1.01426024955437
4,New York City,1,3,1,2015,1.01370567072126
5,New York City,1,4,1,2015,1.01097761785692
6,New York City,1,5,1,2015,1.00681198910082


In [12]:
write.table(df3,'../data/hourwise.txt',sep='|',row.names=F)

In [17]:

######## Plotting starts here#####################
p <-ggplot(df3,aes(day,hour,fill=average_trips))+
  geom_tile(color= "white",size=0.1) + 
  scale_fill_viridis(name='',option ="C")
p <-p + facet_grid(year~month)
p <-p + scale_y_continuous(trans = "reverse", breaks = unique(df3$hour),name = 1:24)
p <-p + scale_x_continuous(breaks =c(1,10,20,31))
p <-p + theme_minimal(base_size = 8)
p <-p + labs(title= paste("Hourly Trips in",statno), x="Day", y="Hour Commencing")
p <-p + theme(legend.position = "bottom")+
        theme(plot.title=element_text(size = 14))+
        theme(axis.text.y=element_text(size=6)) +
        theme(strip.background = element_rect(colour="white"))+
        theme(plot.title=element_text(hjust=0))+
        theme(axis.ticks=element_blank())+
        theme(axis.text=element_text(size=7))+
        theme(legend.title=element_text(size=8))+
        theme(legend.text=element_text(size=6))+
        removeGrid()#ggExtra

In [19]:
svg(filename="../plots/HeatMap.svg", 
    width=24, 
    height=12, 
    pointsize=1)
p
dev.off()

png 
  2

In [ ]:
# you will want to expand your plot screen before this bit!
options(repr.plot.width=4, repr.plot.height=4)
p #awesomeness

#################################